In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
label2id = {'Corrective':0,'Adaptive':1,'Perfective':2}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
# train = train.rename(columns={'3_labels':'label','comment':'text'})
train = train.replace({"label": label2id})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
# test = test.rename(columns={'3_labels':'label','comment':'text'})
test = test.replace({"label": label2id})
test.fillna(0, inplace=True)
print(len(test))



1740


In [4]:
from datasets import Dataset, load_metric

In [5]:
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [6]:
len(train)

53

In [7]:
train['label'].value_counts()

0    18
2    18
1    17
Name: label, dtype: int64

In [8]:
test['label'].value_counts()

0    585
2    582
1    573
Name: label, dtype: int64

In [9]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [10]:
train

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
331,f95ec3f5bf12bee07c90943cff3b135e6a7e7a8b,hadoop,HADOOP-6133. Add a caching layer to- Configura...,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
325,de8f0efe60233436431930447d7672f2a1dc8878,hadoop,MAPREDUCE-3121. NodeManager should handle- dis...,1,0,0,0,1,1,0,...,1,0,1,1,1,1,1,1,0,0
597,14c657d448b6dd743806c4df2a321d58f4e0618e,kotlin,"Extract Function: Consider reference ""broken"" ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
884,6a85160cfa4f99167f29a1323d0175703fb57816,camel,Fixed test--git-svn-id: https://svn.apache.org...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
461,5b4e584e455a1d4713119308c846d66fcfd52d9a,drools,JBRULES-2439--git-svn-id: https://svn.jboss.or...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1780,1f363c2f0b1d5ee8cb7501a2a7a8994349724cc6,apache$hama,Improve message buffering to save memory\ngit-...,2,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
899,ff7d4eebd8ebbf011656313dca8c6ee1a598c2aa,spring-framework,Polish AbstractHandlerMethodMapping---,2,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1157,a57bee46d7db4cd6257d45a0733cf575602b0bb3,tapiji,Creates `stable` branch.\n\nThe stable branch ...,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1386,53fbd0069652cfe101a2eba1c0f0849b35ff28c1,Delta Spike,DELTASPIKE-60 minor JavaDoc tweaks\n,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
846,8f4cb0486b6e6ab48e6df6b2e3e44a7eca27c911,camel,CAMEL-1789 Let the Camel Route support to look...,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [14]:
encoded_train = tokenizer(train['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [15]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [16]:
train_dataset = CommitDataset(encoded_train, list(train['label']))
test_dataset = CommitDataset(encoded_test, list(test['label']))

In [17]:
def compute_metrics(eval_pred):
    accuracy = load_metric('accuracy')
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall.compute(predictions=predictions, references=labels,average='weighted')
    f1 = f1.compute(predictions=predictions, references=labels,average='weighted')
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

<ipython-input-15-ebcd56f8ad98>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.095609,{'precision': 0.2666091844637255},{'recall': 0.3971264367816092},{'f1': 0.3190101968163929},{'accuracy': 0.3971264367816092}


<ipython-input-17-273414206e49>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy = load_metric('accuracy')
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.2666091844637255}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.3971264367816092}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect 

TrainOutput(global_step=4, training_loss=1.1376357078552246, metrics={'train_runtime': 74.853, 'train_samples_per_second': 0.708, 'train_steps_per_second': 0.053, 'total_flos': 6346069522506.0, 'train_loss': 1.1376357078552246, 'epoch': 1.0})